### Import Required Libraries and Set Up Environment Variables

In [1]:
# Dependencies
import requests
import time
from dotenv import load_dotenv
import os
import pandas as pd
import json

In [2]:
# Set environment variables from the .env in the local environment
load_dotenv()

nyt_api_key = os.getenv("NYT_API_KEY")
tmdb_api_key = os.getenv("TMDB_API_KEY")

### Access the New York Times API

In [3]:
# Set the base URL
url = "https://api.nytimes.com/svc/search/v2/articlesearch.json?"

# Filter for movie reviews with "love" in the headline
# section_name should be "Movies"
# type_of_material should be "Review"
filter_query = 'section_name:"Movies" AND type_of_material:"Review" AND headline:"love"'

# Use a sort filter, sort by newest
sort = "newest"

# Select the following fields to return:
# headline, web_url, snippet, source, keywords, pub_date, byline, word_count
field_list = "headline,web_url,snippet,source,keywords,pub_date,byline,word_count"

# Search for reviews published between a begin and end date
begin_date = "20130101"
end_date = "20230531"

# Build URL
query_url = f"{url}api-key={nyt_api_key}&fq={filter_query}&sort={sort}&fl={field_list}&begin_date={begin_date}&end_date={end_date}"


In [4]:
# Create an empty list to store the reviews
reviews = []

# loop through pages 0-19
for page in range(0, 20):
    # create query with a page number
    query_url_page = f"{query_url}&page={page}"
    # API results show 10 articles at a time
    print(f"Retrieving page {page} of results...")
    
    # Make a "GET" request and retrieve the JSON
    response = requests.get(query_url_page).json()
    
    # Add a twelve second interval between queries to stay within API query limits
    time.sleep(12)
    
    # Try and save the reviews to the reviews_list
    try:
        # loop through the reviews["response"]["docs"] and append each review to the list
        for review in response["response"]["docs"]:
            reviews.append(review)

        # Print the page that was just retrieved
        print(f"Page {page} retrieved!")

        # Print the page number that had no results then break from the loop
    except:
        print(f"No results found for page {page}!")
        break


Retrieving page 0 of results...
Page 0 retrieved!
Retrieving page 1 of results...
Page 1 retrieved!
Retrieving page 2 of results...
Page 2 retrieved!
Retrieving page 3 of results...
Page 3 retrieved!
Retrieving page 4 of results...
Page 4 retrieved!
Retrieving page 5 of results...
Page 5 retrieved!
Retrieving page 6 of results...
Page 6 retrieved!
Retrieving page 7 of results...
Page 7 retrieved!
Retrieving page 8 of results...
Page 8 retrieved!
Retrieving page 9 of results...
Page 9 retrieved!
Retrieving page 10 of results...
Page 10 retrieved!
Retrieving page 11 of results...
Page 11 retrieved!
Retrieving page 12 of results...
Page 12 retrieved!
Retrieving page 13 of results...
Page 13 retrieved!
Retrieving page 14 of results...
Page 14 retrieved!
Retrieving page 15 of results...
Page 15 retrieved!
Retrieving page 16 of results...
Page 16 retrieved!
Retrieving page 17 of results...
Page 17 retrieved!
Retrieving page 18 of results...
Page 18 retrieved!
Retrieving page 19 of results...

In [6]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(reviews[:5], indent=4))

[
    {
        "web_url": "https://www.nytimes.com/2023/05/25/movies/the-attachment-diaries-review.html",
        "snippet": "A gynecologist and her patient form a horrifyingly twisted connection in this batty, bloody Argentine melodrama.",
        "source": "The New York Times",
        "headline": {
            "main": "\u2018The Attachment Diaries\u2019 Review: Love, Sick",
            "kicker": null,
            "content_kicker": null,
            "print_headline": "The Attachment Diaries",
            "name": null,
            "seo": null,
            "sub": null
        },
        "keywords": [
            {
                "name": "subject",
                "value": "Movies",
                "rank": 1,
                "major": "N"
            },
            {
                "name": "creative_works",
                "value": "The Attachment Diaries (Movie)",
                "rank": 2,
                "major": "N"
            },
            {
                "name": "persons",
 

200

In [26]:
# Convert reviews_list to a Pandas DataFrame using json_normalize()
reviews_df = pd.json_normalize(reviews)
reviews_df.head()

0      https://www.nytimes.com/2023/05/25/movies/the-...
1      https://www.nytimes.com/2023/05/04/movies/what...
2      https://www.nytimes.com/2023/05/04/movies/you-...
3      https://www.nytimes.com/2023/04/21/movies/a-to...
4      https://www.nytimes.com/2023/04/20/movies/othe...
                             ...                        
195    https://www.nytimes.com/2017/03/09/movies/the-...
196    https://www.nytimes.com/2017/03/09/movies/revi...
197    https://www.nytimes.com/2017/03/02/movies/love...
198    https://www.nytimes.com/2017/02/16/movies/ever...
199    https://www.nytimes.com/2017/02/09/movies/kedi...
Name: web_url, Length: 200, dtype: object


In [29]:
# Extract the title from the "headline.main" column and
# save it to a new column "title"
# Title is between unicode characters \u2018 and \u2019. 
# End string should include " Review" to avoid cutting title early
titles = reviews_df["headline.main"].str.extract(r"\u2018(.*?)\u2019.*?Review")
reviews_df["title"] = titles
reviews_df = reviews_df.dropna(subset=["title"])
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"[{'name': 'subject', 'value': 'Movies', 'rank'...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves


In [30]:
# Extract 'name' and 'value' from items in "keywords" column
def extract_keywords(keyword_list):
    extracted_keywords = ""
    for item in keyword_list:
        # Extract 'name' and 'value'
        keyword = f"{item['name']}: {item['value']};" 
        # Append the keyword item to the extracted_keywords list
        extracted_keywords += keyword
    return extracted_keywords

# Fix the "keywords" column by converting cells from a list to a string
reviews_df["keywords"] = reviews_df["keywords"].apply(extract_keywords)
reviews_df.head()

,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,headline.name,headline.seo,headline.sub,byline.original,byline.person,byline.organization,title
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,None,None,None,By Jeannette Catsoulis,"[{'firstname': 'Jeannette', 'middlename': None...",None,The Attachment Diaries
2,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,You Can Live Forever
3,https://www.nytimes.com/2023/04/21/movies/a-to...,Rachael Leigh Cook stars in this bland rom-com...,The New York Times,subject: Movies;creative_works: A Tourist's Gu...,2023-04-21T07:03:25+0000,276,‘A Tourist’s Guide to Love’ Review: A Wearying...,None,None,A Tourist’s Guide to Love,None,None,None,By Elisabeth Vincentelli,"[{'firstname': 'Elisabeth', 'middlename': None...",None,A Tourist
4,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,None,None,None,By Manohla Dargis,"[{'firstname': 'Manohla', 'middlename': None, ...",None,Other People
5,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,None,None,None,By Brandon Yu,"[{'firstname': 'Brandon', 'middlename': None, ...",None,One True Loves


In [31]:
# Create a list from the "title" column using to_list()
# These titles will be used in the query for The Movie Database
titles_list = reviews_df["title"].to_list()
titles_list

['The Attachment Diaries',
 'You Can Live Forever',
 'A Tourist',
 'Other People',
 'One True Loves',
 'The Lost Weekend: A Love Story',
 'A Thousand and One',
 'Your Place or Mine',
 'Love in the Time of Fentanyl',
 'Pamela, a Love Story',
 'In From the Side',
 'After Love',
 'Alcarràs',
 'Nelly & Nadine',
 'Lady Chatterley',
 'The Sound of Christmas',
 'The Inspection',
 'Bones and All',
 'My Policeman',
 'About Fate',
 'Waiting for Bojangles',
 'I Love My Dad',
 'A Love Song',
 'Alone Together',
 'Art of Love',
 'The Wheel',
 'Thor: Love and Thunder',
 'Both Sides of the Blade',
 'Fire of Love',
 'Love & Gelato',
 'Stay Prayed Up',
 'Benediction',
 'Dinner in America',
 'In a New York Minute',
 'Anaïs in Love',
 'I Love America',
 'See You Then',
 'La Mami',
 'Love After Love',
 'Deep Water',
 'Lucy and Desi',
 'Cyrano',
 'The In Between',
 'Book of Love',
 'Lingui, the Sacred Bonds',
 'The Pink Cloud',
 'A Journal for Jordan',
 'West Side Story',
 'Aulcie',
 'Love Is Love Is Love',

### Access The Movie Database API

In [32]:
# Prepare The Movie Database query
url = "https://api.themoviedb.org/3/search/movie?query="
tmdb_key_string = "&api_key=" + tmdb_api_key

In [34]:
# Create an empty list to store the results
tmdb_movies_list = []

# Create a request counter to sleep the requests after a multiple
# of 50 requests
request_counter = 1


# Loop through the titles
for title in titles_list:

    # Check if we need to sleep before making a request
    if request_counter % 50 == 0 and request_counter != 0:
        print("Sleeping for 1 second...")
        time.sleep(1)

    # Add 1 to the request counter
    request_counter += 1

    # Include a try clause to search for the full movie details.
    # Use the except clause to print out a statement if a movie
    # is not found.
    try:
        # Perform a "GET" request for The Movie Database
        response = requests.get(url + title + tmdb_key_string).json()

        # Get movie id
        movie_id = response["results"][0]["id"]

        # Make a request for a the full movie details
        movie_url = f"https://api.themoviedb.org/3/movie/{movie_id}?api_key={tmdb_api_key}"

        # Execute "GET" request with url
        movie_response = requests.get(movie_url).json()

        # Extract the genre names into a list
        genre_names = [genre["name"] for genre in movie_response["genres"]]

        # Extract the spoken_languages' English name into a list
        spoken_languages = [language['english_name'] for language in movie_response["spoken_languages"]]

        # Extract the production_countries' name into a list
        production_countries = [country['name'] for country in movie_response["production_countries"]]

        # Add the relevant data to a dictionary and
        # append it to the tmdb_movies_list list
        tmdb_movies_list.append({
            "title": movie_response["title"],
            "original_title": movie_response["original_title"],
            "budget": movie_response["budget"],
            "original_language": movie_response["original_language"],
            "homepage": movie_response["homepage"],
            "overview": movie_response["overview"],
            "popularity": movie_response["popularity"],
            "runtime": movie_response["runtime"],
            "revenue": movie_response["revenue"],
            "release_date": movie_response["release_date"],
            "vote_average": movie_response["vote_average"],
            "vote_count": movie_response["vote_count"],
            "genre_names": genre_names,
            "spoken_languages": spoken_languages,
            "production_countries": production_countries,
        })
        
        # Print out the title that was found
        print(f"{title} found!")

    except:
        print(f"{title} not found!")
        continue


The Attachment Diaries found!
You Can Live Forever found!
A Tourist found!
Other People found!
One True Loves found!
The Lost Weekend: A Love Story found!
A Thousand and One found!
Your Place or Mine found!
Love in the Time of Fentanyl found!
Pamela, a Love Story found!
In From the Side found!
After Love found!
Alcarràs found!
Nelly & Nadine found!
Lady Chatterley found!
The Sound of Christmas found!
The Inspection found!
Bones and All found!
My Policeman found!
About Fate found!
Waiting for Bojangles found!
I Love My Dad found!
A Love Song found!
Alone Together found!
Art of Love found!
The Wheel found!
Thor: Love and Thunder found!
Both Sides of the Blade found!
Fire of Love found!
Love & Gelato found!
Stay Prayed Up found!
Benediction found!
Dinner in America found!
In a New York Minute found!
Anaïs in Love found!
I Love America found!
See You Then found!
La Mami found!
Love After Love found!
Deep Water found!
Lucy and Desi found!
Cyrano found!
The In Between found!
Book of Love fou

In [35]:
# Preview the first 5 results in JSON format
# Use json.dumps with argument indent=4 to format data
print(json.dumps(tmdb_movies_list[:5], indent=4))


[
    {
        "title": "The Attachment Diaries",
        "original_title": "El apego",
        "budget": 0,
        "original_language": "es",
        "homepage": "",
        "overview": "Argentina, 1970s. A desperate young woman goes to a clinic to have a clandestine abortion. As her pregnancy is already through the fourth month, the doctor refuses. Instead, she proposes to sell the baby to one of her clients and offers to provide shelter in her house until the child is born. Their disturbed personalities will become intertwined in a strange and dangerous relationship.",
        "popularity": 5.486,
        "runtime": 102,
        "revenue": 0,
        "release_date": "2021-10-07",
        "vote_average": 3.0,
        "vote_count": 4,
        "genre_names": [
            "Drama",
            "Mystery",
            "Thriller",
            "Horror"
        ],
        "spoken_languages": [
            "Spanish"
        ],
        "production_countries": [
            "Argentina"
      

In [36]:
# Convert the results to a DataFrame
tmdb_movies_df = pd.DataFrame(tmdb_movies_list)
tmdb_movies_df.head()

,title,original_title,budget,original_language,homepage,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genre_names,spoken_languages,production_countries
0,The Attachment Diaries,El apego,0,es,,"Argentina, 1970s. A desperate young woman goes...",5.486,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,You Can Live Forever,You Can Live Forever,0,en,https://gooddeedentertainment.com/you-can-live...,"When Jaime, a gay teenager, is sent to live in...",29.347,96,15055,2023-03-24,6.700,33,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,A Tourist's Guide to Love,A Tourist's Guide to Love,0,en,https://www.netflix.com/title/81424906,"After an unexpected break up, a travel executi...",24.422,96,0,2023-04-21,6.300,158,"[Romance, Comedy]","[English, Vietnamese]",[United States of America]
3,Other People,Other People,0,en,,"David, a struggling comedy writer fresh off fr...",17.787,97,93000,2016-09-09,6.300,193,"[Comedy, Drama]",[English],[United States of America]
4,One True Loves,One True Loves,0,en,,Emma and Jesse are living the perfect life tog...,14.097,100,37820,2023-04-07,6.406,69,"[Romance, Drama, Comedy]","[English, Spanish]","[Czech Republic, United States of America]"


### Merge and Clean the Data for Export

In [37]:
# Merge the New York Times reviews and TMDB DataFrames on title
merged_df = pd.merge(reviews_df, tmdb_movies_df, on="title")
merged_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genre_names,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",5.486,102,0,2021-10-07,3.000,4,"[Drama, Mystery, Thriller, Horror]",[Spanish],[Argentina]
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",29.347,96,15055,2023-03-24,6.700,33,"[Drama, Romance]","[English, French]","[Canada, United States of America]"
2,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,"David, a struggling comedy writer fresh off fr...",17.787,97,93000,2016-09-09,6.300,193,"[Comedy, Drama]",[English],[United States of America]
3,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,14.097,100,37820,2023-04-07,6.406,69,"[Romance, Drama, Comedy]","[English, Spanish]","[Czech Republic, United States of America]"
4,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,7.393,95,77145,2023-04-13,6.000,2,[Documentary],[English],[United States of America]


In [38]:
# Remove list brackets and quotation marks on the columns containing lists
# Create a list of the columns that need fixing
columns_to_fix = ["genre_names", "spoken_languages", "production_countries"]


# Create a list of characters to remove
characters_to_remove = ['[', ']', '\'', '"']


# Loop through the list of columns to fix
for column in columns_to_fix:

    # Convert the column to type 'str'
    merged_df[column] = merged_df[column].astype(str)

    # Loop through characters to remove
    for character in characters_to_remove:

        # Remove the character from the column
        merged_df[column] = merged_df[column].str.replace(character, '')


# Display the fixed DataFrame
merged_df.head()


,web_url,snippet,source,keywords,pub_date,word_count,headline.main,headline.kicker,headline.content_kicker,headline.print_headline,...,overview,popularity,runtime,revenue,release_date,vote_average,vote_count,genre_names,spoken_languages,production_countries
0,https://www.nytimes.com/2023/05/25/movies/the-...,A gynecologist and her patient form a horrifyi...,The New York Times,subject: Movies;creative_works: The Attachment...,2023-05-25T11:00:03+0000,295,"‘The Attachment Diaries’ Review: Love, Sick",None,None,The Attachment Diaries,...,"Argentina, 1970s. A desperate young woman goes...",5.486,102,0,2021-10-07,3.000,4,"Drama, Mystery, Thriller, Horror",Spanish,Argentina
1,https://www.nytimes.com/2023/05/04/movies/you-...,Religion comes between two girls falling in lo...,The New York Times,subject: Movies;creative_works: You Can Live F...,2023-05-04T11:00:08+0000,294,‘You Can Live Forever’ Review: Do You Love Me ...,None,None,You Can Live Forever,...,"When Jaime, a gay teenager, is sent to live in...",29.347,96,15055,2023-03-24,6.700,33,"Drama, Romance","English, French","Canada, United States of America"
2,https://www.nytimes.com/2023/04/20/movies/othe...,A radiant Virginie Efira stars as a Parisian t...,The New York Times,"subject: Movies;persons: Zlotowski, Rebecca;cr...",2023-04-20T15:35:13+0000,801,‘Other People’s Children’ Review: True Romance,Critic’s pick,None,Intoxicating Love With a Sobering Turn,...,"David, a struggling comedy writer fresh off fr...",17.787,97,93000,2016-09-09,6.300,193,"Comedy, Drama",English,United States of America
3,https://www.nytimes.com/2023/04/13/movies/one-...,A film adaptation of Taylor Jenkins Reid’s nov...,The New York Times,"subject: Movies;persons: Bracey, Luke (1989- )...",2023-04-13T11:00:06+0000,320,‘One True Loves’ Review: A Romance Lost at Sea,None,None,One True Loves,...,Emma and Jesse are living the perfect life tog...,14.097,100,37820,2023-04-07,6.406,69,"Romance, Drama, Comedy","English, Spanish","Czech Republic, United States of America"
4,https://www.nytimes.com/2023/04/13/movies/the-...,There’s not much Lennon music heard in this do...,The New York Times,subject: Documentary Films and Programs;creati...,2023-04-13T11:00:03+0000,327,‘The Lost Weekend: A Love Story’ Review: When ...,None,None,The Lost Weekend: A Love Story,...,May Pang lovingly recounts her life in rock & ...,7.393,95,77145,2023-04-13,6.000,2,Documentary,English,United States of America


In [41]:
# Drop "byline.person" column
merged_df = merged_df.drop(columns=["byline.person"])


In [42]:
# Delete duplicate rows and reset index
merged_df = merged_df.drop_duplicates().reset_index(drop=True)

In [43]:
# Export data to CSV without the index
merged_df.to_csv("output/nyt_movie_reviews.csv", index=False)
